# 1. Pfad zur Struktur und Orientierung eintragen

In [ ]:
CIF_Dateiname = 'Al2O3_52024.cif'
Orientierung = [0, 0, 1]

# 2. Run

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

import Dans_Diffraction as dif

print(dif.module_info())

In [ ]:
xtl = dif.Crystal(CIF_Dateiname)

print(xtl)

In [ ]:
xtl.Scatter.setup_scatter(wavelength_a = 1.54059295, min_theta = 0, specular = Orientierung)

In [ ]:
xtl.Scatter.print_ref_reflections()

In [ ]:
columns = ['h', 'k', 'l', '2th', 'I']
zeilen = [string.replace(' ', '') for string in [s[1:9] + ',' + s[10:19] + ',' + s[28:] for s in xtl.Scatter.print_ref_reflections().split('\n')[4:-2]]]

data_list = [line.split(',') for line in zeilen]

df = pd.DataFrame(data_list, columns=columns)

df = df.apply(pd.to_numeric, errors='coerce')

df['Struktur'] = CIF_Dateiname
df['Indexsumme'] = df.apply(lambda row: row['h'] + row['k'] + row['l'], axis=1)

df = df.sort_values(by=['I', 'Indexsumme', 'h', '', 'l'], ascending=[False, False, True, True, True])

df['rel I'] = df['I']/df['I'].max()
df['hkl'] = df.apply(lambda row: [row['h'], row['k'], row['l']], axis=1)

df['rho'] = df.apply(lambda row: xtl.Cell.angle(row['hkl'], xtl.Scatter._scattering_specular_direction), axis=1)
df['omega'] = df['2th']/2 - df['rho']

df.head(20)

# 3. Liste der asymetrischen Refelxe, die von oben messbar wären.

In [ ]:
sdf = df[(df['rho'] != 0) & (df['omega'] > 0) & (df['2th'] < 125) & (df['rel I'] > 0.001)]
udf = sdf.drop_duplicates(subset=['2th', 'I'])
head = ['hkl', '2th', 'Struktur', 'rel I', 'rho', 'omega']
udf[head]